In [1]:
import numpy as np
import pandas as pd

from plotly.graph_objs import *
import re

#plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.grid_objs import Grid, Column #sliders
import plotly.graph_objs as go #horizontal bar charts
import plotly.plotly as py
from plotly import tools #side by side
init_notebook_mode(connected=True)

#LDA
import pyLDAvis
import pyLDAvis.gensim
import gensim
from nltk.corpus import stopwords
stop = stopwords.words('english')
pyLDAvis.enable_notebook()
from gensim import corpora, models

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning:

`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.



In [2]:
tweets = pd.read_csv('../../finalprojdata/tweets.csv')
users = pd.read_csv('../../finalprojdata/users.csv')
fulltweets = pd.read_csv('../../finalprojdata/fulltweets2.csv')
fulltweets = pd.read_csv('/Users/shsu/Downloads/fulltweets2.csv')

In [3]:
#clean the date to a Year-Month format
users['Date'] = pd.to_datetime(users['created_at'])
users = users[pd.notnull(users['created_at'])]
users = users.drop_duplicates(subset=['id'])
users['Date'] = users['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [4]:
usersname = pd.DataFrame(users.name.str.split(' ',1).tolist(),
                                   columns = ['first','last'])

usersnamesum = usersname.groupby('first',as_index=False).size().reset_index(name='counts')

usersnamesum = usersnamesum.sort_values('counts', ascending=False).head(20)

## Names: First and Last

In [60]:
## side by side bar plot
#first names
firstnames = usersname.groupby('first',as_index=False).size().reset_index(name='counts')
firstnames = firstnames.sort_values('counts', ascending=False).head(20)
data1 = go.Bar(
            x=firstnames['counts'],
            y=firstnames['first'],
            orientation = 'h',
            name = 'First Name'
)

#lastnames
lastnames = usersname.groupby('last',as_index=False).size().reset_index(name='counts')
lastnames  = lastnames .sort_values('counts', ascending=False).head(20)
data2 = go.Bar(
            x=lastnames ['counts'],
            y=lastnames ['last'],
            orientation = 'h',
            name = 'Last Name'
)

fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('First Name','Last Name'))
fig.append_trace(data1, 1, 1)
fig.append_trace(data2, 1, 2)
fig['layout'].update(height=600, width=800, title='Totals of First and Last Names of Fake Accounts', titlefont=dict(size=25))
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



## Topical Sentiment of User Profile Description

In [6]:
descrip = users.description.copy().astype(str)
descrip = descrip.str.replace('[^\w\s]','')
descrip = descrip.str.replace('[\\r|\\n|\\t|_]',' ')
descrip = descrip.str.strip()

fulltweets_descrip = fulltweets.copy()
fulltweets_descrip.descrip = descrip
fulltweets_descrip.descrip = fulltweets_descrip.descrip.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop)]))

In [7]:
# compile sample documents into a list
doc_set = fulltweets_descrip.descrip.values.copy()
# loop through document list
texts = [text.split(' ') for text in doc_set]
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=30, id2word = dictionary)

In [8]:
data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
data

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
16     5.899010        1       1  0.121391 -0.031886
29     5.635560        1       2  0.051688 -0.148088
3      5.429064        1       3  0.067262 -0.126475
20     5.403172        1       4 -0.069402 -0.023883
12     4.529630        1       5  0.009182  0.122402
10     4.462286        1       6  0.088410  0.073952
0      4.121933        1       7 -0.089567 -0.037511
22     3.951338        1       8 -0.069995  0.011903
9      3.892568        1       9 -0.090962  0.071828
23     3.743412        1      10  0.062808  0.085921
26     3.676319        1      11  0.082024  0.075445
4      3.650585        1      12 -0.080941 -0.013143
14     3.610404        1      13 -0.051074 -0.011184
28     3.540469        1      14 -0.056078 -0.045107
8      3.456818        1      15  0.106314 -0.070028
6      3.431396        1      16  0.089609 -0.099929
2      3.328170        1      17  0.081380  0.054243
25     3.266859        1      18  0.099269  0.086519
13     2.850696        1      19 -0.047932  0.034168
18     2.667612        1      20 -0.049730 -0.020734
27     2.320601        1      21 -0.025343  0.016759
24     2.314718        1      22  0.040118 -0.012711
21     2.132035        1      23  0.045667  0.043258
5      2.093932        1      24 -0.066860 -0.025642
17     2.083896        1      25 -0.054996 -0.005661
11     1.990917        1      26 -0.051539  0.022700
1      1.810305        1      27 -0.023313  0.003737
15     1.705997        1      28 -0.059016 -0.010575
19     1.672399        1      29 -0.062753 -0.009363
7      1.327896        1      30  0.004380 -0.010915, topic_info=     Category       Freq              Term      Total  loglift  logprob
term                                                                   
24    Default  28.000000               nan  28.000000  30.0000  30.0000
39    Default  31.000000              news  31.000000  29.0000  29.0000
153   Default  19.000000      Conservative  19.000000  28.0000  28.0000
585   Default  10.000000                 в  10.000000  27.0000  27.0000
1465  Default   5.000000                HA   5.000000  26.0000  26.0000
44    Default  14.000000  BlackLivesMatter  14.000000  25.0000  25.0000
28    Default  10.000000                не  10.000000  24.0000  24.0000
646   Default  11.000000                 и  11.000000  23.0000  23.0000
104   Default   9.000000               God   9.000000  22.0000  22.0000
879   Default   5.000000               247   5.000000  21.0000  21.0000
886   Default  16.000000                us  16.000000  20.0000  20.0000
127   Default   9.000000              tcot   9.000000  19.0000  19.0000
220   Default   5.000000             world   5.000000  18.0000  18.0000
911   Default   9.000000          Breaking   9.000000  17.0000  17.0000
880   Default  14.000000                DM  14.000000  16.0000  16.0000
59    Default   8.000000              love   8.000000  15.0000  15.0000
163   Default   5.000000             stand   5.000000  14.0000  14.0000
916   Default   6.000000      endorsements   6.000000  13.0000  13.0000
913   Default   6.000000               RTs   6.000000  12.0000  12.0000
915   Default   6.000000           anytime   6.000000  11.0000  11.0000
918   Default   6.000000           weather   6.000000  10.0000  10.0000
917   Default   6.000000           traffic   6.000000   9.0000   9.0000
53    Default   9.000000                Im   9.000000   8.0000   8.0000
4     Default   9.000000            people   9.000000   7.0000   7.0000
465   Default   8.000000          politics   8.000000   6.0000   6.0000
691   Default   5.000000     entertainment   5.000000   5.0000   5.0000
145   Default   6.000000              TCOT   6.000000   4.0000   4.0000
134   Default   4.000000              best   4.000000   3.0000   3.0000
305   Default   6.000000             Black   6.000000   2.0000   2.0000
171   Default   4.000000   Go

## When were the accounts made?

In [9]:
userssum = users.groupby('Date',as_index=False).size().reset_index(name='counts')
userssum = userssum.sort_values('Date')

In [59]:
trace0 = Bar(
    name = "Accounts Created Over Time",
    x=userssum.Date,
    y=userssum.counts)
data = Data([trace0])

layout = Layout(
    title = "Number of Russian Twitter Bots Created Between 2009 and 2017 Per Month",
    yaxis=dict(
        title='Number of Accounts Made',
        range=[0, 100],
        titlefont=dict(
            size=20,
        )
    ),
   
    xaxis = dict(
        title='Year',
        range = ['2009-01','2017-1'],
        titlefont=dict(
            size=20,
        )
    )
)
fig = Figure(data=data, layout=layout)
fig['layout'].update(titlefont=dict(size=25))
iplot(fig)

## Where are they from?

In [11]:
users.location.value_counts()

USA                              93
United States                    23
Москва                           16
Москва, Россия                    9
Chicago, IL                       7
Atlanta                           6
US                                6
Atlanta, GA                       5
New York, NY                      3
Baltimore, MD                     3
Texas, USA                        3
Houston, TX                       3
Санкт-Петербург                   3
Los Angeles, CA                   3
Washington, DC                    2
St Louis, MO                      2
Baton Rouge, LA                   2
Омск                              2
Philadelphia, PA                  2
New York, USA                     2
NY                                2
Мурманск                          2
Miami, FL                         2
Ватноград                         1
Detroit, Michigan                 1
Islamic States of America         1
Brooklyn, NY                      1
Калуга, Россия              

## How influential are these users

In [62]:
trace = go.Scatter(
    x = users.followers_count,
    y = users.statuses_count,
    mode = 'markers',
    text = users.name,
    marker = dict(size = 10,
        color = 'blue',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        ))
)

data = [trace]

layout = Layout(
    title = "Russian Fake Account Followers and Tweet Count",
    xaxis=dict(
            title='Number of Followers',
            dtick=10000,
            range=[0, 100000],
            titlefont=dict(
            size=20,
        )
    ),
   
    yaxis = dict(
        title='Number of Tweets',
        range=[0, 70000],
        titlefont=dict(
            size=20,
        )
    ),
    
    annotations=[dict(y=20576,x=61609,xref='x',
            yref='y',text='Jenna Abrams'),
                dict(y=42438,x=60897,xref='x',
            yref='y',text='New York City Today'),
                dict(y=44882,x=27745,xref='x',
            yref='y',text='New Orleans Online'),
                dict(x=98412,y=30624,xref='x',
            yref='y',text='Максим Дементьев'),
                dict(y=1370,x=11518,xref='x',
            yref='y',text='Black News'),
                dict(y=57212,x=11370,xref='x',
            yref='y',text='РИА ФАН'),
                dict(y=28939,x=31729,xref='x',
            yref='y',text='Washington Online')]
)
fig = Figure(data=data, layout=layout)
fig['layout'].update(height=900, width=1200, titlefont=dict(size=20))
iplot(fig)

## When are they posting?

In [49]:
# heat map showing the days and hours of users
m =pd.pivot_table(fulltweets,values = 'user_key',index='created_strDayofweek',
                      columns='created_strMonth', aggfunc=len,fill_value=0, dropna=False)
z=m.as_matrix()

In [63]:
trace = Heatmap(z=z,
                x=[i for i in np.arange(0,13)],
                y=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday'],
                colorscale = 'Jet'
               )
layout = Layout(
    title='Number of Tweets during the day of the week per hour',
    xaxis=dict(
        nticks=24, 
        title = 'Hour',
        titlefont=dict(
        size=20)),
    
    yaxis = dict(
        title='Day of the Week',
        titlefont=dict(
            size=20,
        )
    ),

)
data=[trace]
fig = Figure(data=data,layout=layout)
fig['layout'].update(titlefont=dict(size=25))
iplot(fig, filename='numberoftweets.html')

## What are they saying?

In [14]:
# LDA topics 
dictionary = gensim.corpora.Dictionary.load('dictionary.dict')
corpus = gensim.corpora.MmCorpus('corpus.mm')
ldamodel = gensim.models.LdaModel.load('topic.model')

In [15]:
data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
data

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
17     5.652078        1       1  0.269390 -0.038418
21     4.082652        1       2  0.207074 -0.011219
28     4.003842        1       3 -0.044191  0.224570
27     3.965532        1       4  0.087466  0.131443
9      3.850252        1       5  0.190262  0.006419
25     3.680442        1       6  0.166511 -0.017891
11     3.526953        1       7 -0.027825  0.066462
14     3.453641        1       8  0.083642 -0.104792
24     3.438111        1       9 -0.027985  0.111117
4      3.321473        1      10 -0.078400  0.164542
0      3.231427        1      11 -0.017994 -0.076229
13     3.230222        1      12  0.058620 -0.002203
23     3.223801        1      13 -0.023846  0.128801
12     3.188437        1      14 -0.003502  0.105461
7      3.184742        1      15 -0.043826 -0.092018
19     3.098035        1      16 -0.062619 -0.077927
8      3.086067        1      17 -0.027137 -0.051963
29     3.080491        1      18 -0.049124 -0.019002
3      3.076198        1      19  0.050652  0.003056
16     3.033868        1      20 -0.097865 -0.055391
10     3.015934        1      21 -0.080050 -0.095464
22     3.012110        1      22 -0.038976 -0.059701
20     3.003440        1      23 -0.085357 -0.049674
5      2.999063        1      24 -0.072599 -0.006369
15     2.984936        1      25 -0.050851 -0.035763
1      2.951488        1      26 -0.080618 -0.057125
18     2.946492        1      27 -0.084379  0.010517
2      2.922456        1      28 -0.059199 -0.082429
6      2.913232        1      29 -0.059057  0.003384
26     2.842588        1      30  0.001783 -0.022193, topic_info=       Category          Freq             Term         Total  loglift  logprob
term                                                                          
235874  Default  23817.000000            trump  23817.000000  30.0000  30.0000
31953   Default   9612.000000          clinton   9612.000000  29.0000  29.0000
172447  Default   9021.000000          hillary   9021.000000  28.0000  28.0000
38966   Default   4989.000000             dont   4989.000000  27.0000  27.0000
48151   Default   3588.000000             tcot   3588.000000  26.0000  26.0000
56046   Default   4473.000000           donald   4473.000000  25.0000  25.0000
27477   Default   3921.000000         politics   3921.000000  24.0000  24.0000
183588  Default   3521.000000          america   3521.000000  23.0000  23.0000
10757   Default   4863.000000             like   4863.000000  22.0000  22.0000
36914   Default   3093.000000        president   3093.000000  21.0000  21.0000
238384  Default   2893.000000             know   2893.000000  20.0000  20.0000
135144  Default   2797.000000             vote   2797.000000  19.0000  19.0000
22149   Default   2611.000000            never   2611.000000  18.0000  18.0000
233129  Default   2505.000000             good   2505.000000  17.0000  17.0000
119324  Default   2670.000000             want   2670.000000  16.0000  16.0000
223723  Default   2392.000000            white   2392.000000  15.0000  15.0000
136538  Default   2599.000000            pjnet   2599.000000  14.0000  14.0000
74981   Default   2432.000000             love   2432.000000  13.0000  13.0000
66034   Default   2584.000000             maga   2584.000000  12.0000  12.0000
176039  Default   2247.000000         midnight   2247.000000  11.0000  11.0000
231205  Default   2439.000000            think   2439.000000  10.0000  10.0000
7915    Default   2240.000000             cant   2240.000000   9.0000   9.0000
136102  Default   7366.000000            obama   7366.000000   8.0000   8.0000
30154   Default   2033.000000             back   2033.000000   7.0000   7.0000
35015   Default   1938.000000         american   1938.000000   6.0000   6.0000
190213  Default   4277.000000             news   4277.000000   5.0000   5.0000
216484  Default   2735.000000             make   

# How are they gaining attention?

In [16]:
subusers = fulltweets.groupby('user_key', as_index=False).mean().sort_values(by='followers_count', ascending=False).user_key[0:20].values

In [17]:
subfulltweets2 = fulltweets[fulltweets.user_key.isin(subusers)][['user_key', 'followers_count', 'Subjectivity', 'Sentiment', 'statuses_count', 'created_str']]

In [18]:
subfulltweets2['Date'] = pd.to_datetime(subfulltweets2['created_str'])
subfulltweets2 = subfulltweets2[pd.notnull(subfulltweets2['created_str'])]
subfulltweets2['Date'] = subfulltweets2['Date'].apply(lambda x: x.strftime('%Y-%m')) #change to '%Y-%m-%d' for fine tuned

## Tweet Velocity

In [19]:
tweetcount = subfulltweets2.groupby('Date',as_index=False).size().reset_index(name='counts')
tweetcount = tweetcount.sort_values('Date', ascending=True)


In [71]:
trace0 = Scatter(
    name = "Tweets Made",
    x=tweetcount.Date,
    y=tweetcount.counts)
data = Data([trace0])

layout = Layout(
    title = "Number of Tweets by Top 20 Followed Bot Accounts",
    yaxis=dict(
        title='Number of Tweets',
        range=[0, 3500],
        titlefont=dict(
            size=20,
        )
    ),
   
    xaxis = dict(
        title='Year',
        range = ['2014-12','2017-5'],
        titlefont=dict(
            size=20,
        )
    )
)
fig = Figure(data=data, layout=layout)
fig['layout'].update(titlefont=dict(size=20))
iplot(fig)

## Tweet Sentiment

In [21]:
tweetsent = subfulltweets2.groupby('Date',as_index=False).sum()
tweetsent = tweetsent.sort_values('Date', ascending=True)


In [70]:
trace0 = Scatter(
    name = "Sentiment Over Time",
    x=tweetsent.Date,
    y=tweetsent.Sentiment)
data = Data([trace0])

layout = Layout(
    title = "Sentiment by Top 20 Followed Bot Accounts",
    yaxis=dict(
        title='Sum of Sentiments',
        range=[-10,100],
        titlefont=dict(
            size=20,
        )
    ),
   
    xaxis = dict(
        title='Year',
        range = ['2014-12','2017-5'],
        titlefont=dict(
            size=20,
        )
    )
)
fig = Figure(data=data, layout=layout)
fig['layout'].update(titlefont=dict(size=20))
iplot(fig)

## Tweet Subjectivity

In [69]:
trace0 = Scatter(
    x=tweetsent.Date,
    y=tweetsent.Subjectivity)
data = Data([trace0])

layout = Layout(
    title = "Subjectivity by Top 20 Followed Bot Accounts",
    yaxis=dict(
        title='Sum of Subjectivity',
        range=[-20, 800],
        titlefont=dict(
            size=20,
        )
    ),
   
    xaxis = dict(
        title='Year',
        range = ['2014-12','2017-5'],
        titlefont=dict(
            size=20,
        )
    )
)
fig = Figure(data=data, layout=layout)
fig['layout'].update(titlefont=dict(size=20))
iplot(fig)

## Combined Graphs

In [68]:
trace0 = Scatter(
    name = "Sentiment Over Time",
    x=tweetsent.Date,
    y=tweetsent.Sentiment)

trace1 = Scatter(
    name = "Subjectivity Over Time",
    x=tweetsent.Date,
    y=tweetsent.Subjectivity)

data = Data([trace0, trace1])

layout = Layout(
    title = "Aggregation of Top 20 Followed Bot Account Metrics",
    yaxis=dict(
        title='Value',
        range=[-20, 800],
        titlefont=dict(
            size=20,
        )
    ),
   
    xaxis = dict(
        title='Year',
        range = ['2014-12','2017-5'],
        titlefont=dict(
            size=20,
        )
    )
)
fig = Figure(data=data, layout=layout)
fig['layout'].update(titlefont=dict(size=20))
iplot(fig)